# card tagger

simple enough

1. generate a dropdown of all cards
1. select a dropdown to update an image of the card and get the text
1. tag shit

In [1]:
import sys, os

sys.path.insert(0, os.path.realpath('../'))

In [2]:
import functools
import logging

import requests

from IPython.display import Image, display
from ipywidgets import Button, Checkbox, Dropdown, HTML, Output, SelectMultiple, Text

from neo4j import basic_auth, GraphDatabase

from mtg.cards import cards_df
from mtg.credentials import F_NEO_CONF, load_neo_config
from mtg.extract.tappedout import get_all_categories
from mtg.load.nx2neo import digraph_to_neo, verify_constraints
from mtg.utils import init_logging

## logging

In [3]:
LOGGER = logging.getLogger('card tagger')
LOGGER.setLevel(logging.DEBUG)

In [4]:
init_logging()

## basic functionality

In [5]:
@functools.lru_cache(None)
def get_card_info():
    return (cards_df()
            .groupby('name')
            .last()
            .reset_index()
            [['name', 'multiverseId']]
            .dropna())

In [6]:
@functools.lru_cache(None)
def get_image_url(multiverseId):
    url = 'https://api.scryfall.com/cards/multiverse/{}'.format(int(multiverseId))
    return requests.get(url).json()['image_uris']['normal']

In [7]:
KNOWN_NODES_QRY = "MATCH (n:Tag) RETURN DISTINCT n.name AS name"

def get_neo_tags(neo_conf):
    neo4juri = 'bolt://{ip}:{port}'.format(**neo_conf)
    auth = basic_auth(neo_conf['user'], neo_conf['pw'])
    with GraphDatabase.driver(neo4juri, auth=auth) as driver:
        with driver.session() as session:
            LOGGER.info('loading all known tags from {}'.format(neo4juri))
            resp = session.run(KNOWN_NODES_QRY)
            known_tags = {_['name'] for _ in resp.data()}
            
            return known_tags

In [8]:
PUBLISH_TAGS_QRY = """MATCH (c:Card {name: $card_name})
MATCH (t:Tag {name: $tag_name})
MERGE (c)-[:HAS_TAG]->(t)"""

def publish_tags(neo_conf, card_name, tag_names):
    neo4juri = 'bolt://{ip}:{port}'.format(**neo_conf)
    auth = basic_auth(neo_conf['user'], neo_conf['pw'])
    with GraphDatabase.driver(neo4juri, auth=auth) as driver:
        with driver.session() as session:
            LOGGER.info('publishing the provided tags to {}'.format(neo4juri))
            for tag_name in tag_names:
                LOGGER.info('({})-[:HAS_TAG]->({})'.format(card_name, tag_name))
                resp = session.run(PUBLISH_TAGS_QRY,
                                   card_name=card_name,
                                   tag_name=tag_name)
                LOGGER.info(resp.data())

## building the display widgets

In [9]:
cards = get_card_info()

widget shit

In [10]:
f_neo_conf = F_NEO_CONF
tapped_out_owner='ndlambo'

neo_conf = load_neo_config(f_neo_conf)

neo_tags = get_neo_tags(neo_conf)

2019-07-25 22:33:48,219 INFO     [card tagger.get_neo_tags:8] loading all known tags from bolt://localhost:7687


In [11]:
# del card_name_selector, tag_name_selector, card_image, publish_button

In [12]:
card_name_selector = Dropdown(options=cards.values.tolist())
tag_name_selector = Dropdown(options=sorted(neo_tags))
card_image = HTML("select card name to load")
publish_button = Button(description="publish'em")

In [13]:
cards.head()

,name,multiverseId
0,"""Ach! Hans, Run!""",73935.0
1,"""Rumors of My Death . . .""",439454.0
3,A Display of My Dark Power,212578.0
4,A Reckoning Approaches,430661.0
5,AWOL,74231.0


In [14]:
def update_image(change):
    """when the card name has changed, update the image"""
    try:
        png_uri = get_image_url(change['new'])
        card_image.value = '<img src="{}" width="300">'.format(png_uri)
    except Exception as e:
        LOGGER.info('unable to load image for change to {}'.format(change['new']))
        card_image.value = 'select different card name'

card_name_selector.observe(update_image, names='value')

def publish_on_click(b):
    """publish shit to neo when a button has been clicked"""
    card_name = cards[cards.multiverseId == card_name_selector.value].name.iloc[0]
    publish_tags(neo_conf, card_name, [tag_name_selector.value])

publish_button.on_click(publish_on_click)

In [15]:
display(card_name_selector,
        card_image,
        tag_name_selector,
        publish_button)

Dropdown(options=(['"Ach! Hans, Run!"', 73935.0], ['"Rumors of My Death . . ."', 439454.0], ['A Display of My …

HTML(value='select card name to load')

Dropdown(options=('+1/+1 Counter Source', '06-06-18-esper-blink - Amplify', '06-06-18-esper-blink - Engine', '…

Button(description="publish'em", style=ButtonStyle())